In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import matplotlib.pyplot as plt
import time
import os
import random
import io
from PIL import Image, ImageOps
from keras.models import Sequential, Model
import keras.layers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Input
from keras.regularizers import l2
from keras.optimizers import Adam, Adadelta, RMSprop
import keras.losses as losses
from keras.models import load_model
from keras.backend import set_image_data_format
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import glob

set_image_data_format('channels_first')

Using TensorFlow backend.


#### Parameters

In [2]:
STILL_ALIVE_REWARD = 1
DEAD_REWARD = -1000

CROP_SHAPE = (750, 539, 1)
RESIZE_WIDTH = 180
RESIZE_HEIGHT = 137
READ_BATCH = 2

MOVES = 5
NONE = 0
RIGHT = 1
DOWN = 2
LEFT = 3
UP = 4

SHOTS_FOLDER = 'data/shots/'

SAMPLE_ALIVE = 1
SAMPLE_DEAD = 5
SAMPLE_REWARD = 20

MAX_MEMORY = 500

GAMMA = 0.75
EPS = 0.1

#### Browser functions

In [3]:
def selectLevel(l):
    xpath = '/html/body/section/div[2]/nav/p[' + str(l+1) + ']'
    
    level = browser.find_element_by_xpath(xpath)
    level.click()
    
    
def clickBoard():        
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

    try:
        board.click()
    except WebDriverException:
        print('Exception')
        return
    
    
def saveScreen(fileName):    
    state = getState()
#     if state == 'playing' or state == 'paused':    
    ss = browser.get_screenshot_as_file(SHOTS_FOLDER + fileName)
    
    
def getScreen():
    ss = browser.get_screenshot_as_png()
    im = Image.open(io.BytesIO(ss))
    
    im = preprocImg(im)
    return np.asarray(im.convert("L"))
    
    
def getState():
    xpath = '/html/body/section/div[2]'
    state = browser.find_element_by_xpath(xpath)
    c = state.get_attribute('class')
    
    return c.split(' ')[-1]


def getScore():
    state = getState()
    
    if state == 'playing' or state == 'paused':
        
        xpath = '/html/body/section/div[2]/p[1]/span'
        score = browser.find_element_by_xpath(xpath)

        if not score.text.isnumeric():
            return 0
        return int(score.text)
    
    return 0


def makeMove(m):
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

    if m == RIGHT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_RIGHT)
    elif m == DOWN:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_DOWN)
    elif m == LEFT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_LEFT)
    elif m == UP:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_UP)
        

def makeRandomMove():
    r = random.randint(0, 4)
    makeMove(r)
    
    return r

#### Preprocess functions

In [4]:
def getLastGameIndex(folder):
    '''Get the index of the last current game.'''
    files = os.listdir(folder)
    if len(files) == 0:
        return 0
    
    last = files[-1]
    
    return parseName(last)[0]


def getName(g, i, m, r):
    '''Form the file name from the board data.'''
    return str(g).zfill(3) + '_' + str(i).zfill(3) + '_' + str(m) + '_' + str(r) + '.png'


def parseName(fileName):
    '''Parse the board data from the file name.'''
    return [int(d) for d in fileName[:-4].split('_')]


def saveImage(arr, file):
    '''Save an image array to a file'''
    img = Image.fromarray(arr.astype(np.uint8))
    img.save(file)


def cropImgOld(fileName):
    ss = plt.imread(fileName)

    ss = ss[120:659, 273:1023, :]
    
    plt.imsave(fileName, ss)
        

def preprocImg(im):
    '''Preprocess a screenshot.'''     
    # Crop board
    im = im.crop((273, 120, 1023, 659))
    
    # Grayscale
    im = ImageOps.grayscale(im)
    
    # Binarization
    t = 127
    im = im.point(lambda x: 255 if x > t else 0)
    
    # Resize
    im = im.resize((RESIZE_WIDTH, RESIZE_HEIGHT))
    
    return im
    
    
def preprocAll(gameIndex):
    '''Preprocess all screenshots in the data folder.'''
    files = os.listdir(SHOTS_FOLDER)
    for f in files:
        g = parseName(f)[0]
        
        if g >= gameIndex:        
            fileName =  SHOTS_FOLDER + f
            im = Image.open(fileName)   
            im = preprocImg(im)
            im.save(fileName)

### Q model

#### Input functions

In [5]:
def readData():
    files = os.listdir(SHOTS_FOLDER)
    
    labels = np.zeros(len(files))    
    data = []
    
    for i, f in enumerate(files):
        im = Image.open(SHOTS_FOLDER + f)
        
        if f in history:
            labels[i] = history[f][2]
        else:
            labels[i] = STILL_ALIVE_REWARD
            
        data.append([np.asarray(im.convert("L"))])
        
    data = np.asarray(data)
    return data, labels


def sampleData(nSamples, 
               propD=SAMPLE_DEAD, 
               propR=SAMPLE_REWARD,
               propA=SAMPLE_ALIVE):
    '''Sample the input data according to some proportions.
    
    Parameters
    ----------
    nSamples: total number of samples
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    files = os.listdir(SHOTS_FOLDER)
    
    perc = []
    nd, na, nr = (0, 0, 0)
    for f in files:
        r = parseName(f)[3]
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(files, size=nSamples, replace=False, p=perc)
    print('Sampled data.')
    return samples


def readSamplesBasic(samples):
    '''Read samples for basic model, which just classifies if a board is finished or not.'''
    data = []
    labels = []
    
    for file in samples:
        im = Image.open(SHOTS_FOLDER + file)
        g, i, m, r = parseName(file)
        
        data.append([np.asarray(im.convert("L"))])
        labels.append(1 if r > 0 else 0)
        
    data = np.asarray(data)
    return data, labels


def getPreviousSample(file):
    '''Get the name of the previous sample.'''
    
    g, i, m, r = parseName(file)
    i -= 1
    
    if i > 0:        
        prevFile = glob.glob(SHOTS_FOLDER + str(g).zfill(3) + '_' + str(i).zfill(3) + '*')[0]
        prevFile = prevFile.replace('\\', '/')
        prevFile = prevFile.split('/')[-1]
        return prevFile      
    else:
        return -1


def getNextSample(file):
    '''Get the name of the next sample.'''
    
    g, i, m, r = parseName(file)
    i += 1
    
    prevFile = glob.glob(SHOTS_FOLDER + str(g).zfill(3) + '_' + str(i).zfill(3) + '*')
    if len(prevFile) == 1:
        prevFile = prevFile[0]
        prevFile = prevFile.replace('\\', '/')
        prevFile = prevFile.split('/')[-1]
        return prevFile
    else:
        return -1
    
    
def readSample(file):
    '''Read a single sample as an array.'''
    im = Image.open(SHOTS_FOLDER + file)
    g, i, m, r = parseName(file)

    data = np.asarray(im.convert("L"))
    label = 1 if r > 0 else 0
    
    return data, label
    
    
def readSampleMove(file):
    '''Read a single sample as an array, also returns the move.'''
    im = Image.open(SHOTS_FOLDER + file)
    g, i, m, r = parseName(file)

    data = np.asarray(im.convert("L"))
    if r == -1000:
        label = -1
    elif r > 1:
        label = r / 100
    else:
        label = 0
#     label = r
    
    return data, m, label


def readBatchSamples(samples, batch):
    '''Read samples in batches, so that we also have information about the movement.'''
    data = []
    labels = []
    
    for file in samples:        
        dataBatch = []
        
        d, label = readSample(file)
        dataBatch.append(d)
        
        # Get previous files
        prevFile = file
        for p in range(batch - 1):
            prevFile = getPreviousSample(prevFile)
            
            if prevFile == -1:
                break
                
            d, _ = readSample(prevFile)
            dataBatch.append(d)
        
        if prevFile != -1:
            dataBatch.reverse()
            data.append(dataBatch)
            labels.append(label)
        
    data = np.asarray(data)
    return data, labels   


def getStateBatches(file, batch):
    '''Get the image batches for the current and next state.'''
          
    s, _, _ = readSampleMove(file)
    
    # Get the image batch for the current state
    curBatch = []
    curBatch.append(s)
    
    # Get previous files
    prevFile = file
    for p in range(batch - 1):
        prevFile = getPreviousSample(prevFile)
        
        if prevFile == -1:
            break
            
        s, _ = readSample(prevFile)
        curBatch.append(s)
        
    if prevFile == -1:
        return -1, -1
    else:
        #Get the image batch for the next state
        nextBatch = []
        
        nextFile = getNextSample(file)
        if nextFile == -1:
            curBatch.reverse()
            return curBatch, curBatch
        
        nextState, _ = readSample(nextFile)        
        
        nextBatch.append(nextState)        
        nextBatch.extend(curBatch[:-1])
        
        curBatch.reverse()
        nextBatch.reverse()
        return curBatch, nextBatch


def readBatchSamplesMove(samples, batch):
    '''Read samples in batches, so that we also have information about the movement.
    Also reads the moves.'''
    states = []
    actions = []
    labels = []
    nextStates = []
    
    for i, file in enumerate(samples):          
        _, a, r = readSampleMove(file)
        
        curBatch, nextBatch = getStateBatches(file, batch)
        if curBatch != -1:
            states.append(curBatch)
            actions.append(a)
            labels.append(r)
            nextStates.append(nextBatch)

        if i % 100 == 0:
            print(i)
        
    states = np.asarray(states)
    actions = np.asarray(actions)
    nextStates = np.asarray(nextStates)
    labels = np.asarray(labels)
       
    # Scale labels
#     labels = np.asarray(labels).reshape(-1, 1)
#     scaler = StandardScaler()
#     scaler.fit(labels)
#     labels = scaler.transform(labels)
    
    return states, actions, labels, nextStates    
    
    
def getModelMoveInput(model, screens):
    '''Get the move from the model which takes moves as input.'''
    
    m = 0
    bestMove = 0
    bestScore = DEAD_REWARD - 1
    
    while m < MOVES:
        score = model.predict([np.asarray([screens]), np.asarray([m])])      
        
        print(m, score)
        if score > bestScore:
            bestScore = score
            bestMove = m
            
        m += 1        
    
    
def getModelMoveOutput(model, screens):
    '''Get the move from the model which produces move scores as outputs.'''
    res = model.predict(np.asarray([screens]))
    m = np.argmax(res)
    
    return m     
    
    
def getMoveOutputEps(model, screens, eps):
    '''Get the move from the model which produces move scores as outputs, with an epsilon probability for exploration.'''
    
    # If there is a model
    if model != False:        
        p = random.random()
        
        # If exploitation was chosen
        if p > eps:
            res = model.predict(np.asarray([screens]))
            m = np.argmax(res)
            print('Model:', m, end='\r')

            return m
   
    # If there is no model or exploration was chosen
    m = random.randint(0, 4) 
    print('Rando:', m, end='\r')
    
    return m 
    

#### DNN action output

Functional API, citeste batch-uri de imagini, considera actiunea ca output. 

In [6]:

def nn_model_actionoutput():
    '''Full NN model, using the Keras Functional API.
    Action as output, final layer has as many neurons as possible actions.'''
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv1')(screen_input)
    conv = Conv2D(16, (4, 4), activation='relu', name='conv2')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool1')(conv)
    
    conv = Conv2D(8, (4, 4), activation='relu', name='conv3')(pool)
    conv = Conv2D(8, (4, 4), activation='relu', name='conv4')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool2')(conv)
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv5')(pool)
    pool = MaxPool2D(pool_size=(2,2), name='pool3')(conv)
    
    flat = Flatten(name='flat')(pool)
    dense = Dense(100, activation='relu', name='dense1')(flat)
    drop = Dropout(rate=0.2)(dense)
    dense = Dense(40, activation='relu', name='dense2')(drop)
    drop = Dropout(rate=0.2)(dense)
    
    output = Dense(MOVES, activation='linear', name='output')(drop)
    
    model = Model(inputs=screen_input, outputs=output)
    
    model.compile(loss='mean_squared_error',
                 optimizer = 'adam',
                 metrics=['accuracy'])
    
    return model

#### Train model function

In [7]:

class Experience:
    '''Class for an experience.'''
    def __init__(self, state, action, label, nextState):
        self.state = state
        self.action = action
        self.label = label
        self.nextState = nextState
                        
            
def rememberExp(memory, exps):
    '''Add new experiences to the memory.'''
    for e in exps:
        if len(memory) == MAX_MEMORY:
            memory.pop(0)
        memory.append(e)


def sampleMemory(memory, nSamples, propD, propR, propA):
    '''Sample experiences from the memroy according to some proportions.
    
    Parameters
    ----------
    memory: the remembered experiences
    nSamples: total number of samples
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    nSamples = min(len(memory), nSamples)
        
    perc = []
    nd, na, nr = (0, 0, 0)
    for exp in memory:
        r = exp.label
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(memory, size=nSamples, replace=False, p=perc)
    return samples


def evaluateLabels(model, states, actions, labels, sample=0):
    
    if sample > 0:
        sample = min(sample, len(states))
        indexes = random.sample(range(len(states)), sample)
        states = states[indexes]
        actions = actions[indexes]
        labels = labels[indexes]
        
    res = model.predict(states)   
    
    sqErr = [(res[i][actions[i]] - labels[i])**2 for i, out in enumerate(res)]
    meanSqErr = np.mean(sqErr)
    
    return meanSqErr


def trainModelOnline(memory, 
                     model,
                     batchSize=32, 
                     file='data/models/model.h5',
                     status=True
                     ):
    '''Train a model online, from the experience memory. 
    Can load or generate a new model. Saves the model at the end.
    
    Parameters
    ----------
    memory: the experience memory
    model: the model to be trained
    batchSize: the training batch size
    file: the model file, used for saving
    status: if true, print the status of the training
    '''
        
    # Sample experiences from the memory
    batch = sampleMemory(memory, batchSize, 5, 10, 1)
    batchSize = len(batch)

    states = []
    targets = []
    # For each experience
    for e in batch:        
        target = e.label
        if e.label != DEAD_REWARD:
            target = e.label + GAMMA * np.amax(model.predict(np.asarray([e.nextState])))
        
        fullTarget = model.predict(np.asarray([e.state]))
        fullTarget[0][e.action] = target
        
        targets.append(fullTarget[0])
        states.append(e.state)
               
    # Fit the model with the batch
    targets = np.asarray(targets)
    model.fit(np.asarray(states), targets, epochs=1, verbose=0)
        
#     print('MSE =', evaluateLabels(model, states, actions, labels, sample=50))
    model.save(file)
                
    return model


def trainModelOffline(states, actions, labels, nextStates, 
                      epochs, 
                      batchSize=32, 
                      file='data/models/model.h5', 
                      load=False,
                      status=True
                     ):
    '''Train a model offline. 
    Can load or generate a new model. Saves the model after each epoch.
    
    Parameters
    ----------
    states: the list of batches of images representing the current states
    actions: the list of actions taken
    labels: the list of rewards obtained
    nextStates: the list of batches of images representing the next states
    epochs: the number of training epochs
    batchSize: the training batch size
    file: the model file, used for saving and loading
    load: if true, the model is loaded from the file, otherwise a new model is generated
    status: if true, print the status of the training
    '''
    
    if load:
        model = load_model(file)  
        print('Model loaded.')
    else:
        model = nn_model_actionoutput()

    dataSize = len(states)
    
    # For each epoch
    for i in range(epochs):
        if status:
            print('Epoch', i+1)
        
        time1 = time.time()
    
        batchStart = 0
        batchEnd = min(batchSize, dataSize)
    
        # For each batch
        while batchStart < batchEnd:

            targets = []
            
            b = batchStart
            # For each experience of the batch, recalculate the targets according to the Q algorithm
            while b < dataSize and b < batchEnd:
                # Get the features of the experience
                s = states[b : b+1]
                a = actions[b]
                l = labels[b]
                n = nextStates[b: b+1]
                
                target = l
                if l != DEAD_REWARD:
                    target = l + GAMMA * np.amax(model.predict(n))
                
                fullTarget = model.predict(s)
                fullTarget[0][a] = target
                
                targets.append(fullTarget[0])
                
                b += 1
            
            if status:
                print(batchStart, '/', dataSize, end='\r')
                
            # Fit the model with the batch
            targets = np.asarray(targets)
            model.fit(states[batchStart:batchEnd], targets, epochs=1, verbose=0)

            batchStart = batchEnd
            batchEnd = min(batchEnd + batchSize, dataSize)
          
        if status:  
            print(dataSize, '/', dataSize)
            print("  MSE = %.2f" % evaluateLabels(model, states, actions, labels, sample=50))
        model.save(file)
        
        time2 = time.time()
        
        if status:
            print("  Duration: %.2f s" % (time2- time1))
        
        
    return model

In [9]:
# states, actions, labels, nextStates = readBatchSamplesMove(sampleData(6000), READ_BATCH)
# s2, a2, l2, n2 = readBatchSamplesMove(sampleData(50), READ_BATCH)
# print(states.shape)

model = trainModelOffline(states, actions, labels, nextStates, epochs=50, load=True, file='data/models/model_s6000_e211.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Model loaded.
Epoch 1
5445 / 5445
  MSE = 0.03
  Duration: 296.68 s
Epoch 2
5445 / 5445
  MSE = 0.04
  Duration: 270.71 s
Epoch 3
5445 / 5445
  MSE = 0.01
  Duration: 270.19 s
Epoch 4
5445 / 5445
  MSE = 0.01
  Duration: 267.81 s
Epoch 5
5445 / 5445
  MSE = 0.02
  Duration: 270.38 s
Epoch 6
5445 / 5445
  MSE = 0.01
  Duration: 269.30 s
Epoch 7
5445 / 5445
  MSE = 0.02
  Duration: 271.92 s
Epoch 8
5445 / 5445
  MSE = 0.01
  Duration: 273.06 s
Epoch 9
5445 / 5445
  MSE = 0.02
  Duration: 271.82 s
Epoch 10
5445 / 5445
  MSE = 0.01
  Duration: 273.97 s
Epoch 11
5445 / 5445
  MSE = 0.04
  Duration: 273.49 s
Epoch 12
5445 / 5445
  MSE = 0.02
  Duration: 275.54 s
Epoch 13
5445 / 5445
  MSE = 0.01
  Duration: 275.36 s
Epoch 14
5445 / 5445
  MSE = 0.01
  Duration:

### Play game

In [8]:
def play(games,
         train=False,
         useModel=False,
         saveImages=False,
         modelFile='data/models/model.h5',
         saveFolder=SHOTS_FOLDER):
    global browser
    
    e = EPS
    
    url = 'https://playsnake.org/'
#     browser = webdriver.Chrome(executable_path='D:/Libraries/Drivers/chromedriver_win32/chromedriver.exe')
    browser = webdriver.Chrome(executable_path='C:/Personal/Proiecte/Cod/_libs/Python/selenium_chromedriver73_win32/chromedriver.exe')
    browser.get(url) 

    START_GAME = getLastGameIndex(saveFolder) + 1
    gameIndex = START_GAME
    memory = []
    
    # Load model
    if useModel == True:
        curModel = load_model(modelFile)
    elif train == True:
        curModel = nn_model_actionoutput()
    else:
        curModel = False

    for g in range(games):

        # Select the level
        selectLevel(1)
        time.sleep(2)

        score = 0
        screenIndex = 0
        time1 = time.time()

        clickBoard()
        prevScreen = getScreen()
    
        state = []
        episode = []        
        finalScore = 0
        
        while getState() == 'playing' or getState() == 'paused':            
            curScreen = getScreen()       
            screenIndex += 1    
            
            # Save the current screenshot
            if saveImages:
                saveImage(curScreen, saveFolder + 'prev.png')

            # Add the new experience to the episode
            nextState = [prevScreen, curScreen]
            if state and nextState:
                exp = Experience(state, m, r, nextState)
                episode.append(exp)
            state = nextState
        
            # Make the move            
            m = getMoveOutputEps(curModel, state, e)
        
            clickBoard()
            makeMove(m)
            clickBoard()

            # Calculate the reward
            r = max(STILL_ALIVE_REWARD, getScore() - score) 
        
            # Update last board  
            if saveImages:
                fileName = getName(gameIndex, screenIndex, m, r)
                os.rename(saveFolder + 'prev.png', saveFolder + fileName)
            
            score = getScore()
            finalScore = max(finalScore, score)

            prevScreen = curScreen

        # Change reward of last board
        if saveImages:
            g, i, m, r = parseName(fileName)
            os.rename(saveFolder + fileName, saveFolder + getName(g, i, m, DEAD_REWARD))

    
        if train:            
            # Add the episode to the memory
            rememberExp(memory, episode)

            # Retrain the model
            curModel = trainModelOnline(memory, model=curModel, file=modelFile)
        else:
            time.sleep(1)            
            
        gameIndex += 1
        if e > MIN_EPS:
            e = e * 0.995
        
        time2 = time.time()        
        print("%d: %.2f s, %d points" % (g, time2-time1, finalScore))

    browser.quit()



In [9]:
# Train online
EPS = 1.0
MIN_EPS = 0.05
MAX_MEMORY = 1000
play(3000, 
#      useModel=True, 
     modelFile='data/models/model_o3000.h5', 
     train=True   
    )

# Save images
# play(1, saveImages=True)

# Play with model
# EPS = 0.0
# play(1, useModel=True, modelFile='data/models/model_s3000_e11.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
0: 19.16 s, 0 points
1: 10.40 s, 0 points
2: 33.49 s, 0 points
3: 35.60 s, 0 points
4: 80.95 s, 5 points
5: 83.18 s, 0 points
6: 20.14 s, 0 points
7: 19.36 s, 0 points
8: 19.39 s, 0 points
9: 16.01 s, 0 points
10: 62.24 s, 0 points
11: 12.32 s, 0 points
12: 18.18 s, 0 points
13: 11.96 s, 0 points
14: 30.65 s, 0 points
15: 72.88 s, 0 points
16: 19.78 s, 0 points
17: 86.23 s, 0 points
18: 17.74 s, 0 points
19: 10.40 s, 0 points
20: 36.60 s, 0 points
21: 12.72 s, 0 points
22: 13.46 s, 0 points
23: 17.27 s, 0 points
24: 16.63 s, 0 points
25: 10.01 s, 0 points
26: 53.58 s, 0 points
27: 16.55 s, 20 points
28: 27.18 s, 0 points
29: 14.82 s, 0 points
30: 8.99 s, 0 points
31: 16.46 s, 0 points
32: 11.72 s, 0 points
33: 23.51 s, 0 points
34: 15.75 s, 0 points
35: 3

322: 10.42 s, 0 points
323: 11.69 s, 0 points
324: 9.99 s, 0 points
325: 11.27 s, 0 points
326: 13.42 s, 0 points
327: 9.68 s, 0 points
328: 10.95 s, 0 points
329: 10.83 s, 0 points
330: 12.27 s, 0 points
331: 9.49 s, 60 points
332: 9.63 s, 0 points
333: 8.92 s, 0 points
334: 10.61 s, 0 points
335: 11.18 s, 0 points
336: 12.37 s, 0 points
337: 10.37 s, 0 points
338: 9.36 s, 0 points
339: 10.68 s, 0 points
340: 10.70 s, 0 points
341: 11.49 s, 0 points
342: 28.73 s, 10 points
343: 11.50 s, 0 points
344: 10.12 s, 0 points
345: 9.73 s, 0 points
346: 9.31 s, 0 points
347: 10.76 s, 0 points
348: 10.45 s, 0 points
349: 10.36 s, 0 points
350: 10.75 s, 0 points
351: 11.55 s, 0 points
352: 11.36 s, 0 points
353: 10.19 s, 0 points
354: 11.69 s, 0 points
355: 10.79 s, 0 points
356: 11.64 s, 0 points
357: 11.22 s, 0 points
358: 9.20 s, 0 points
359: 11.13 s, 0 points
360: 10.24 s, 0 points
361: 9.22 s, 0 points
362: 11.67 s, 0 points
363: 10.28 s, 0 points
364: 14.25 s, 0 points
365: 11.17 s, 0 poi

680: 10.03 s, 65 points
681: 10.07 s, 0 points
682: 10.67 s, 75 points
683: 11.44 s, 0 points
684: 10.46 s, 0 points
685: 10.58 s, 0 points
686: 11.13 s, 0 points
687: 9.77 s, 0 points
688: 11.99 s, 0 points
689: 11.08 s, 0 points
690: 10.09 s, 0 points
691: 10.95 s, 0 points
692: 10.86 s, 0 points
693: 10.28 s, 0 points
694: 13.06 s, 0 points
695: 10.34 s, 0 points
696: 10.55 s, 0 points
697: 10.12 s, 0 points
698: 11.47 s, 0 points
699: 15.68 s, 65 points
700: 10.03 s, 0 points
701: 10.91 s, 0 points
702: 9.56 s, 0 points
703: 10.13 s, 0 points
704: 13.49 s, 0 points
705: 11.34 s, 0 points
706: 15.56 s, 0 points
707: 11.30 s, 0 points
708: 17.23 s, 0 points
709: 15.56 s, 0 points
710: 14.19 s, 0 points
711: 10.76 s, 0 points
712: 17.90 s, 0 points
713: 13.18 s, 0 points
714: 9.92 s, 0 points
715: 11.59 s, 0 points
716: 10.77 s, 0 points
717: 11.91 s, 0 points
718: 10.09 s, 0 points
719: 12.10 s, 0 points
720: 10.20 s, 0 points
721: 10.72 s, 0 points
722: 11.77 s, 0 points
723: 9.66 s

1037: 9.74 s, 0 points
1038: 9.48 s, 0 points
1039: 10.72 s, 0 points
1040: 9.26 s, 65 points
1041: 9.23 s, 0 points
1042: 9.39 s, 0 points
1043: 11.29 s, 0 points
1044: 8.15 s, 0 points
1045: 17.56 s, 0 points
1046: 10.11 s, 0 points
1047: 10.82 s, 0 points
1048: 10.65 s, 0 points
1049: 10.73 s, 0 points
1050: 9.71 s, 0 points
1051: 10.39 s, 0 points
1052: 11.47 s, 0 points
1053: 10.26 s, 0 points
1054: 10.19 s, 0 points
1055: 10.32 s, 0 points
1056: 9.98 s, 0 points
1057: 11.80 s, 0 points
1058: 10.35 s, 0 points
1059: 8.95 s, 0 points
1060: 10.33 s, 0 points
1061: 11.23 s, 0 points
1062: 10.78 s, 0 points
1063: 10.44 s, 0 points
1064: 11.43 s, 0 points
1065: 12.06 s, 0 points
1066: 10.74 s, 0 points
1067: 11.21 s, 0 points
1068: 10.44 s, 0 points
1069: 11.24 s, 0 points
1070: 12.64 s, 0 points
1071: 8.66 s, 0 points
1072: 10.43 s, 0 points
1073: 11.23 s, 0 points
1074: 10.51 s, 0 points
1075: 10.84 s, 0 points
1076: 10.84 s, 95 points
1077: 9.24 s, 0 points
1078: 10.14 s, 0 points
1

1381: 9.93 s, 0 points
1382: 9.55 s, 0 points
1383: 10.96 s, 0 points
1384: 9.89 s, 0 points
1385: 10.18 s, 0 points
1386: 11.64 s, 0 points
1387: 10.99 s, 0 points
1388: 10.67 s, 80 points
1389: 8.37 s, 0 points
1390: 10.67 s, 0 points
1391: 9.90 s, 0 points
1392: 10.65 s, 0 points
1393: 9.82 s, 0 points
1394: 11.37 s, 0 points
1395: 9.83 s, 0 points
1396: 10.68 s, 0 points
1397: 11.20 s, 0 points
1398: 11.38 s, 0 points
1399: 10.15 s, 0 points
1400: 11.02 s, 0 points
1401: 9.24 s, 0 points
1402: 10.76 s, 0 points
1403: 9.91 s, 0 points
1404: 10.34 s, 0 points
1405: 10.57 s, 0 points
1406: 11.21 s, 0 points
1407: 10.24 s, 0 points
1408: 9.46 s, 80 points
1409: 11.20 s, 0 points
1410: 11.37 s, 0 points
1411: 10.42 s, 0 points
1412: 12.46 s, 0 points
1413: 10.03 s, 0 points
1414: 10.54 s, 0 points
1415: 10.30 s, 0 points
1416: 9.25 s, 0 points
1417: 10.01 s, 0 points
1418: 9.29 s, 0 points
1419: 10.03 s, 0 points
1420: 9.62 s, 0 points
1421: 11.13 s, 0 points
1422: 9.17 s, 0 points
1423

1726: 10.44 s, 0 points
1727: 11.33 s, 0 points
1728: 12.55 s, 35 points
1729: 10.79 s, 0 points
1730: 10.68 s, 0 points
1731: 8.51 s, 0 points
1732: 10.45 s, 0 points
1733: 9.89 s, 0 points
1734: 11.57 s, 0 points
1735: 10.18 s, 0 points
1736: 10.95 s, 0 points
1737: 10.62 s, 0 points
1738: 10.88 s, 0 points
1739: 11.00 s, 0 points
1740: 10.29 s, 0 points
1741: 10.41 s, 0 points
1742: 11.17 s, 0 points
1743: 10.27 s, 0 points
1744: 10.30 s, 0 points
1745: 11.43 s, 0 points
1746: 10.95 s, 0 points
1747: 9.87 s, 0 points
1748: 11.29 s, 0 points
1749: 9.88 s, 0 points
1750: 10.93 s, 0 points
1751: 11.02 s, 0 points
1752: 11.94 s, 0 points
1753: 12.42 s, 0 points
1754: 9.92 s, 0 points
1755: 10.88 s, 0 points
1756: 10.53 s, 0 points
1757: 12.46 s, 0 points
1758: 9.04 s, 0 points
1759: 10.23 s, 0 points
1760: 12.05 s, 0 points
1761: 11.52 s, 0 points
1762: 9.44 s, 0 points
1763: 11.56 s, 0 points
1764: 10.91 s, 0 points
1765: 10.51 s, 0 points
1766: 12.38 s, 0 points
1767: 10.94 s, 0 point

2071: 9.34 s, 0 points
2072: 10.57 s, 0 points
2073: 10.74 s, 0 points
2074: 10.09 s, 0 points
2075: 12.62 s, 0 points
2076: 11.05 s, 0 points
2077: 10.50 s, 0 points
2078: 9.64 s, 0 points
2079: 10.63 s, 50 points
2080: 8.33 s, 0 points
2081: 9.81 s, 0 points
2082: 8.48 s, 0 points
2083: 9.25 s, 0 points
2084: 10.10 s, 0 points
2085: 10.53 s, 0 points
2086: 10.58 s, 70 points
2087: 10.88 s, 0 points
2088: 12.09 s, 0 points
2089: 9.98 s, 0 points
2090: 9.86 s, 0 points
2091: 11.74 s, 0 points
2092: 9.38 s, 0 points
2093: 11.38 s, 0 points
2094: 9.38 s, 0 points
2095: 10.69 s, 0 points
2096: 9.04 s, 0 points
2097: 9.94 s, 0 points
2098: 11.19 s, 0 points
2099: 10.61 s, 0 points
2100: 11.59 s, 0 points
2101: 10.36 s, 0 points
2102: 11.75 s, 0 points
2103: 10.70 s, 0 points
2104: 9.35 s, 0 points
2105: 11.20 s, 0 points
2106: 10.86 s, 0 points
2107: 10.70 s, 0 points
2108: 10.16 s, 0 points
2109: 11.75 s, 0 points
2110: 10.08 s, 0 points
2111: 10.67 s, 0 points
2112: 9.74 s, 0 points
2113

KeyboardInterrupt: 

### Evaluation

In [ ]:
# Build pipeline

layers = []
layers.append(('nn', KerasClassifier(build_fn=nn_model_actionlast,
                                   epochs=5,
                                   batch_size=10,
                                   verbose=1)))
estimator = Pipeline(layers)


# Evaluate

data, actions, labels = readBatchSamplesMove(sampleData(200), 2)

labels = np.asarray(labels)
labels = labels.reshape((labels.size, 1))
actions = actions.reshape((actions.size, 1))
print(data.shape)
print(actions.shape)
print(labels.shape)
     
sKFold = StratifiedKFold(n_splits=2)
results = cross_val_score(estimator, [data, actions], labels, cv=sKFold, verbose=1, scoring='accuracy')

print("Results: %.5f (%.5f)" % (results.mean(), results.std()))